# 2.2 Data preprocessing

自定义一个数据集，存在../data/house_tiny.csv文件中，创建属性，写入数据

In [3]:
import os
import pandas as pd

os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    # 列名
    f.write('NumRooms,Alley,Price\n')
    # 每行表示一个数据样本
    f.write('NA,Pave,127500\n')
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

## 2.2.1 读取数据集

In [4]:
data = pd.read_csv(data_file)
print(data)

   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106000
2       4.0   NaN  178100
3       NaN   NaN  140000


## 2.2.2 处理缺失值
inputs和outputs分别通过使用索引方法`iloc`来区分开，前两列为inputs，最后一列为outputs。inputs中缺少的数值，使用均值代替。在这里，NumRooms是可以计算均值的，所以NaN部分就被NumRooms这一列的均值代替。

In [8]:
inputs = data.iloc[:, 0:2]
outputs = data.iloc[:, 2]
inputs['NumRooms'] = inputs['NumRooms'].fillna(inputs['NumRooms'].mean())
print(inputs)

   NumRooms Alley
0       3.0  Pave
1       2.0   NaN
2       4.0   NaN
3       3.0   NaN


对于inputs中的类别值或离散值，我们将“NaN”视为一个类别。 由于“巷子类型”（“Alley”）列只接受两种类型的类别值“Pave”和“NaN”， pandas可以自动将此列转换为两列“Alley_Pave”和“Alley_nan”。   
这里其实就是在做**one-hot encoding**。

In [11]:
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

   NumRooms  Alley_Pave  Alley_nan
0       3.0        True      False
1       2.0       False       True
2       4.0       False       True
3       3.0       False       True


## 2.2.3 转换为张量格式

In [16]:
import tensorflow as tf

# 这里也可以用constant
X = tf.convert_to_tensor(inputs.to_numpy(dtype=float))
Y = tf.convert_to_tensor(outputs.to_numpy(dtype=float))
X, Y

(<tf.Tensor: shape=(4, 3), dtype=float64, numpy=
 array([[3., 1., 0.],
        [2., 0., 1.],
        [4., 0., 1.],
        [3., 0., 1.]])>,
 <tf.Tensor: shape=(4,), dtype=float64, numpy=array([127500., 106000., 178100., 140000.])>)

这里有一个数据类型转换的要点：PyTorch 和 TensorFlow 都无法直接从 Pandas 的 DataFrame / Series 创建张量，必须先转成 NumPy。


```
# 以torch为例
X = torch.tensor(inputs.to_numpy(), dtype=torch.float32)
y = torch.tensor(outputs.to_numpy(), dtype=torch.float32)

# 以tensor为例
X = tf.convert_to_tensor(inputs.to_numpy(dtype=float))
Y = tf.convert_to_tensor(outputs.to_numpy(dtype=float))
```



## 2.2.5 练习
创建包含更多行和列的原始数据集。
- 删除缺失值最多的列。
- 将预处理后的数据集转换为张量格式。

In [28]:
os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file2 = os.path.join('..', 'data', 'house_tiny2.csv')
with open(data_file2, 'w') as f:
  f.write('NumRooms,Alley,City,Price,\n')
  f.write('NA,Pave,NA,127500,\n')
  f.write('2,NA,NA,106000,\n')
  f.write('4,NA,NA,178100,\n')
  f.write('NA,NA,NA,140000,\n')

data2 = pd.read_csv(data_file2)
data2

,NumRooms,Alley,City,Price,Unnamed: 4
0,NaN,Pave,NaN,127500,NaN
1,2.0,NaN,NaN,106000,NaN
2,4.0,NaN,NaN,178100,NaN
3,NaN,NaN,NaN,140000,NaN


删除全是NaN的整列

In [33]:
data2.dropna(axis=1, how='all')

,NumRooms,Price,Alley_Pave,Alley_nan
0,NaN,127500,True,False
1,2.0,106000,False,True
2,4.0,178100,False,True
3,NaN,140000,False,True


将预处理后的数据集转换为张量格式

In [34]:
# 因为存在Alley这种string类型的元素所以需要进行one hot encoding 独热编码
data2 = pd.get_dummies(data2, dummy_na=True)
data2

,NumRooms,City,Price,Unnamed: 4,Alley_Pave,Alley_nan
0,NaN,NaN,127500,NaN,True,False
1,2.0,NaN,106000,NaN,False,True
2,4.0,NaN,178100,NaN,False,True
3,NaN,NaN,140000,NaN,False,True


In [35]:
data2 = tf.convert_to_tensor(data2.to_numpy(dtype=float))
data2

<tf.Tensor: shape=(4, 6), dtype=float64, numpy=
array([[      nan,       nan, 1.275e+05,       nan, 1.000e+00, 0.000e+00],
       [2.000e+00,       nan, 1.060e+05,       nan, 0.000e+00, 1.000e+00],
       [4.000e+00,       nan, 1.781e+05,       nan, 0.000e+00, 1.000e+00],
       [      nan,       nan, 1.400e+05,       nan, 0.000e+00, 1.000e+00]])>